In [1]:
using Plots
using Statistics
using Surrogates
using CSV
using DataFrames
using Distributions
using Flux
import JSON
using MLUtils
using Plots
using StatsBase
using StatsPlots
using Tables

include("../../scan_hyperparameters_withcellnum_functions.jl")

main (generic function with 1 method)

### Loading data and making cuts for proper scaling

In [2]:
target_directory = "../../data/full_with_cellnumber/"

println("Formatting data...")
x_raw_df, y_df, cellnumber_df = getrawdata_withcellnum(target_directory)

# how to process cellnumber, for now let's make it another dvar
x_raw_df = hcat(x_raw_df, cellnumber_df)

# cutting
println("Cutting Transmission to 60-100 percent...")
lower::Float32 = 60
upper::Float32 = 120
x_raw_df, y_df = applycut(x_raw_df, y_df, "OBJ1", lower, upper; with_numcells=true)

# decorrelating
println("Decorrelating...")
x_df = decorrelatedvars(x_raw_df; with_numcells=true)

x_scaled_df, x_scalers = minmaxscaledf(x_df);
y_scaled_df, y_scalers = minmaxscaledf(y_df);

println("Done!")

Formatting data...
Cutting Transmission to 60-100 percent...
x_raw shape: (217293, 15)
y_raw shape: (217293, 6)
x_new shape: (161574, 15)
y_new shape: (161574, 6)
Decorrelating...
Done!


### Loading up previously trained model

In [3]:
@load "../../models/2023-05-09_09-30-08_w=100_d=5_activation=sigmoid_bs=1024_lr=0.001_dr=0.0_5.jld2" m;

```
DVAR1: Bmax  [ 8.5, 12.0 ]
DVAR2: mX1  [ 5, 140 ]
DVAR3: mX2  [ 15, 160 ]
DVAR4: mY1  [ 1.005, 1.7 ]
DVAR5: mY2  [ 1.055, 1.85 ]
DVAR6: mtau1  [ 1, 500 ]
DVAR7: mtau2  [ 1, 500 ]
DVAR8: PhiY1  [ -89.95, -30 ]
DVAR9: PhiY2  [ -87.45, -25 ]
DVAR10: Phitau1  [ 1, 500 ]
DVAR11: Phitau2  [ 1, 500 ]
DVAR12: mY3ref  [ 1.105, 2.0 ]
DVAR13: PhiY3ref  [ -84.95, -20 ]
DVAR14: Eref  [ 0.055, 0.075 ]
```

### Sampling for init of acquisition function

In [4]:
# bounds of DVARS
lower_bound = repeat([-1.], 14); upper_bound = repeat([1.], 14)

# smapling to instantiate acquisition function
n_samples = 1000
initial_samples = Surrogates.sample(n_samples, lower_bound, upper_bound, SobolSample());

In [5]:
# sampling of odd/even cell number
cellnumber_samples = Float64.(rand(Bernoulli(0.5), n_samples) .* 2 .- 1);
initial_samples_with_cellnumber = []
for i in 1:n_samples
    push!(initial_samples_with_cellnumber, (initial_samples[i]..., cellnumber_samples[i]))
end

In [6]:
# decorrelating design variables
function getdvarprimesing(data_to_transform, dynamic_lower_bound, strict_upper_bound)
    (data_to_transform - dynamic_lower_bound) / (strict_upper_bound - dynamic_lower_bound)
end

function decorr(vec)
    maxes = Dict(
        "DVAR3"=>160.,
        "DVAR5"=>1.85,
        "DVAR9"=>-25.,
        "DVAR12"=>2.,
        "DVAR13"=>-20.
    )
    etas = Dict(
        "DVAR3"=>10.,
        "DVAR5"=>0.05,
        "DVAR9"=>2.5,
        "DVAR12"=>0.05,
        "DVAR13"=>2.5
    )
    
    decorred_vec = []
    
    for i in 1:14
        if i == 3
            append!(decorred_vec, getdvarprimesing(vec[3], etas["DVAR3"] + vec[2], maxes["DVAR3"]))
        elseif i == 5
            append!(decorred_vec, getdvarprimesing(vec[5], etas["DVAR5"] + vec[4], maxes["DVAR5"]))
        elseif i == 9
            append!(decorred_vec, getdvarprimesing(vec[9], etas["DVAR9"] + vec[8], maxes["DVAR9"]))
        elseif i == 12
            append!(decorred_vec, getdvarprimesing(vec[12], etas["DVAR12"] + vec[5], maxes["DVAR12"]))
        elseif i == 13
            append!(decorred_vec, getdvarprimesing(vec[13], etas["DVAR13"] + vec[9], maxes["DVAR13"]))
        else
            append!(decorred_vec, vec[i])
        end
    end
    
    decorred_vec
end

function getvarsing(data_to_transform, dynamic_lower_bound, strict_upper_bound)
    dynamic_lower_bound + data_to_transform*(strict_upper_bound - dynamic_lower_bound)
end

function recorr(vec)
    maxes = Dict(
        "DVAR3"=>160.,
        "DVAR5"=>1.85,
        "DVAR9"=>-25.,
        "DVAR12"=>2.,
        "DVAR13"=>-20.
    )
    etas = Dict(
        "DVAR3"=>10.,
        "DVAR5"=>0.05,
        "DVAR9"=>2.5,
        "DVAR12"=>0.05,
        "DVAR13"=>2.5
    )
    dvarmap = Dict(
        3=>2,
        5=>4,
        9=>8,
        12=>5,
        13=>9
    )
    
    recorred_vec = []
    
    for i in 1:14
        if i ∈ [3, 5, 9]
            append!(recorred_vec, getvarsing(vec[i], etas["DVAR$i"] + vec[dvarmap[i]], maxes["DVAR$i"]))
        elseif i ∈ [12, 13]
            append!(recorred_vec, getvarsing(vec[i], etas["DVAR$i"] + recorred_vec[dvarmap[i]], maxes["DVAR$i"]))
        else
            append!(recorred_vec, vec[i])
        end
    end
    
    recorred_vec
end

function minmaxscale(vec; var="x", inverse=false)
    scaled_vec = []
    
    transform_fun(scaler, x) = inverse ? inverse_transform(scaler, x) : transform(scaler, x)
    
    if var == "x"
        for i in 1:14
            scaler = x_scalers["DVAR$i"]
            append!(scaled_vec, transform_fun(scaler, vec[i]))
        end
    elseif var == "y"
        for i in 1:6
            scaler = y_scalers["OBJ$i"]
            append!(scaled_vec, transform_fun(scaler, vec[i]))
        end
    end
    
    return Tuple(scaled_vec)
end

function predict(x_vec)
    m([x for x in x_vec])'
end

predict (generic function with 1 method)

In [15]:
# do i understand how the model predicts for a given x_vec?
println(first(x_scaled_df))
println(values(first(x_scaled_df)))

println("Prediction:")
println(predict(values(first(x_scaled_df))))
println("True:")
println(first(y_scaled_df))

DataFrameRow
 Row │ DVAR1     DVAR10     DVAR11    DVAR12    DVAR13    DVAR14     DVAR2      DVAR3     DVAR4     DVAR5     DVAR6      DVAR7      DVAR8      DVAR9     cellnumber 
     │ Any       Any        Any       Any       Any       Any        Any        Any       Any       Any       Any        Any        Any        Any       Any        
─────┼──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ 0.289202  -0.447758  0.404934  0.171099  0.357681  -0.434087  -0.313747  0.147541  0.668876  0.553479  -0.784712  -0.816006  -0.451444  0.331384  -1.0
(0.28920162f0, -0.44775814f0, 0.4049338f0, 0.17109923841804942, 0.3576814045090202, -0.43408716f0, -0.3137467f0, 0.1475414222175786, 0.6688763f0, 0.5534791401182384, -0.7847122f0, -0.8160064f0, -0.45144355f0, 0.3313838669893703, -1.0f0)
Prediction:
[0.5421043518299267 -0.4837618505197945 -0.47046635609522464 -0.47913499524829156 0.1

This means that the DVARs used as inputs to the model are out of order -- they don't run from DVAR1 to DVAR14, but instead run in alphabetical order, with cellnumber as a column appended in the end.

### Functionality to predict cellnumber

In [11]:
first(x_scaled_df)

,DVAR1,DVAR10,DVAR11,DVAR12,DVAR13,DVAR14,DVAR2,DVAR3
,Any,Any,Any,Any,Any,Any,Any,Any
1,0.289202,-0.447758,0.404934,0.171099,0.357681,-0.434087,-0.313747,0.147541


### Final optimization
- Maximize transmission (< 95%)
- Output energy should be 70 KeV (0.070 MeV)
- Minimize RFQ length
- Minimize longitudinal emittance
- Minimize transverse emittance

In [29]:
function objective3(x_vec)
    y = predict(x_vec)
    target_energy_scaled = transform(y_scalers["OBJ2"], 0.070)[1]
    [-y[1], (y[2] - target_energy_scaled)^2, y[3], y[4], y[5], y[6]]
end

objective3 (generic function with 1 method)

In [30]:
x3s = [sample for sample in initial_samples_with_cellnumber];
x3s = unique(x3s)
y3s = objective3.(x3s);

In [32]:
if length(x3s) != length(unique(x3s))
    print("Duplicate entries in x3")
end
if length(x3s) != length(y3s)
    print("Dimension mismatch between x3 and y")
end
radial_basis_ego_3 = RadialBasis(x3s, y3s, repeat([-1.], 15), repeat([1.], 15))

(::RadialBasis{Surrogates.var"#1#2", Int64, Vector{NTuple{15, Float64}}, Vector{Vector{Float64}}, Vector{Float64}, Vector{Float64}, Matrix{Float64}, Float64, Bool}) (generic function with 1 method)

In [35]:
pareto_set_3, pareto_front_3 = surrogate_optimize(
    objective3, SMB(), repeat([-1.], 15), repeat([1.], 15), radial_basis_ego_3, SobolSample();
    maxiters = 100, n_new_look = 100
);

In [36]:
obj1_3 = [inverse_transform(y_scalers["OBJ1"], -item[1])[1] for item in pareto_front_3];
dE_3 = [item[2] for item in pareto_front_3];
obj3_3 = [inverse_transform(y_scalers["OBJ3"], item[3])[1] for item in pareto_front_3];
obj4_3 = [inverse_transform(y_scalers["OBJ4"], item[4])[1] for item in pareto_front_3];
obj5_3 = [inverse_transform(y_scalers["OBJ5"], item[5])[1] for item in pareto_front_3];
obj6_3 = [inverse_transform(y_scalers["OBJ6"], item[6])[1] for item in pareto_front_3];

unscaled_pareto_front_3 = [obj1_3, dE_3, obj3_3, obj4_3, obj5_3, obj6_3]

6-element Vector{Vector{Float64}}:
 [63.90773682744614, 95.11441217259939, 96.00102484509253, 78.4978539172133, 65.2214162844422, 74.94593354046272, 68.68760943475563, 76.77794719654288, 55.812625647163635, 72.73669134311868  …  54.33054144506285, 57.09682906729952, 97.02513730213025, 55.953655258266835, 67.0458079807602, 57.54955260320244, 54.98440742731742, 61.45785754439095, 54.57009492883476, 58.518338893434866]
 [0.19417073157308737, 1.6312522980583302, 2.2373417092635286, 1.4948699694607173, 2.12485302304525, 1.2837304426223062, 1.2883649540256248, 0.8504104991332044, 0.36155244760090177, 0.47797529744472605  …  0.2817110047843377, 1.7295394315714017, 2.3685681959719265, 0.28152524163144316, 1.1520891861621052, 2.2483050440176204, 2.002347973583141, 2.1457874424614216, 2.352763212970679, 2.6793586564423095]
 [0.060235249562297505, 0.06265204024141866, 0.06813862999109634, 0.07302493411459171, 0.061572085727646386, 0.05714624606834008, 0.07465697899062922, 0.05991215051003009, 0.0

In [37]:
emit_cutoff = 0.04
long_emit_mask = obj4_3 .< emit_cutoff; trans_emit_mask = obj5_3 .< emit_cutoff;


transmission_cutoff = 90
high_transmission_mask = obj1_3 .> 90

sum(long_emit_mask .& trans_emit_mask .& high_transmission_mask)

6

In [38]:
candidates = [i for i in 1:length(obj1_3)][long_emit_mask .& trans_emit_mask .& high_transmission_mask]

6-element Vector{Int64}:
 111
 168
 261
 303
 332
 353

In [59]:
test_x_transformed = minmaxscale(
    decorr([
        11.312972997315228, 103.74379149079323, 149.3844627470438,
        1.4468421232886612, 1.5137411917386079, 140.85678020119667,
        473.1951289474964, -50.19836366921663, -43.642034931217914,
        245.1396713256836, 245.14222234487534, 1.8956206666091737,
        -37.693693497970166, 0.06037112592457561,
    ]);
    var="x"
)

println(predict((test_x_transformed..., 0.)))
println(predict((test_x_transformed..., 1.)))

[0.7679930472923572 0.3380752670733307 -0.45640557252933145 -0.8362856137674357 -0.7435289543839798 -0.7734108778689787]
[0.7866750186403212 0.3188235154034012 -0.4554849642937827 -0.826415742873267 -0.8345830277685841 -0.6575148567804928]


In [39]:
for j in candidates
    println("candidate $j")
    candidate_dvars = minmaxscale(pareto_set_3[j]; var="x", inverse=true)
    println(recorr(candidate_dvars))
    for i in 1:6
        println("OBJ$i: ", unscaled_pareto_front_3[i][j])
    end
    println(" ")
end

candidate 111
Any[11.312972997315228, 103.74379149079323, 149.3844627470438, 1.4468421232886612, 1.5137411917386079, 140.85678020119667, 473.1951289474964, -50.19836366921663, -43.642034931217914, 245.1396713256836, 245.14222234487534, 1.8956206666091737, -37.693693497970166, 0.06037112592457561]
OBJ1: 95.28683161814637
OBJ2: 2.809695989088585
OBJ3: 0.06304837006663246
OBJ4: 0.03855834786956841
OBJ5: 0.024661802831386033
OBJ6: 0.03333013285786453
 
candidate 168
Any[9.146016827784479, 64.984718054533, 76.2309538327026, 1.505210397299379, 1.790983624151413, 159.37423488497734, 287.04686722159386, -49.37874283641577, -44.208072484098636, 446.882479429245, 415.69542080163956, 1.9007704867739823, -38.63378069219178, 0.07181638060865225]
OBJ1: 95.79925478867364
OBJ2: 1.6381989708262623
OBJ3: 0.06562014177207598
OBJ4: 0.03444770569187049
OBJ5: 0.028652583536487843
OBJ6: 0.023653367517351667
 
candidate 261
Any[9.544203568715602, 125.43041591346264, 136.066503835052, 1.1370151804294437, 1.794

Clearly, `138` is the winner.

In [41]:
candidate_dvars[[1, 2, 3]]

(9.14601474441588, 64.9847204387188, 0.014658961890745559)

In [121]:
function plot_pairwise_scatter(vec1, vec2, vec1label, vec2label, indexes, winner_index, incl_xticks, incl_yticks)
    loser_indexes = [i for i in indexes if i != winner_index]
    
    
    scatter(
        [vec1[i] for i in loser_indexes], [vec2[i] for i in loser_indexes];
        xlabel=vec1label, ylabel=vec2label, legend=false, markersize=3, markerstrokewidth=0.5,
        xformatter = incl_xticks ? :auto : Returns(""), yformatter = incl_yticks ? :auto : Returns("") 
    )
    
    scatter!([vec1[winner_index]], [vec2[winner_index]]; markershape=:star5, markersize=10, markerstrokewidth=0.5)
end

plot_pairwise_scatter (generic function with 3 methods)

In [124]:
labels = [
    "Transmission [%]",
    "Scaled target/output\nenergy squared error",
    "RFQ length [cm]",
    "Longitudinal emittance\n[MeV deg]",
    "Transverse (radial)\nemittance [cm mrad]"
]
indexes = [i for i in 1:length(pareto_front_3)]

layout = @layout([ ° _ _ _ ; ° ° _ _ ; ° ° ° _ ; ° ° ° ° ])

#plot(p1, p2, p3, layout = l)

plots_dict = Dict()

for i in 1:5
    for j in 1:5
        if i < j
            p = plot_pairwise_scatter(
                unscaled_pareto_front_3[i], unscaled_pareto_front_3[j],
                j==5 ? labels[i] : "", i==1 ? labels[j] : "", indexes, 138, j==5, i==1
            )
            if i == 1
                xflip!(true)
            end
            # display(p)
            merge!(plots_dict, Dict("$i$j"=>p))
        end
    end
end

pareto_plot = plot(
    [plots_dict["$i$j"] for j in 1:5 for i in 1:5 if i<j]...,
    layout = layout, widths=(1/4,1/4,1/4,1/4), size=(1000,800), dpi=500
)

# savefig(pareto_plot,"../../img/pareto_plot.png")

## CHECKING OPTIMAL ANSWER FROM ABOVE

In [30]:
optimal_dvars = [
    9.14601474441588,
    64.9847204387188,
    76.2309561819387,
    1.5052128004608676,
    1.7909850165139312,
    159.3742377460003,
    287.0468702018261,
    -49.37874272093177,
    -44.20807974211494,
    446.882483959198,
    415.69541972875595,
    1.9007708828235987,
    -38.63379833969139,
    0.07181640870840056,
]

optimal_dvars_decorr = decorr(optimal_dvars);
optimal_dvars_decorred_scaled = minmaxscale(optimal_dvars_decorr);
optimal_predicted_objs_scaled = predict(optimal_dvars_decorred_scaled);
optimal_predicted_objs = minmaxscale(optimal_predicted_objs_scaled; var="y", inverse=true)

(95.67445783296316, 0.06584600251118666, 147.91082243803984, 0.035490431684761965, 0.03885549528570143, 0.77829399178314)